In [4]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import yfinance as yf
import talib as ta

# Fetch historical data
data = yf.download('AAPL', start='2015-01-01', end='2023-01-01')

# Calculate indicators
data['SMA50'] = ta.SMA(data['Close'], timeperiod=50)
data['SMA200'] = ta.SMA(data['Close'], timeperiod=200)
data['RSI'] = ta.RSI(data['Close'], timeperiod=14)
data['UpperBB'], data['MiddleBB'], data['LowerBB'] = ta.BBANDS(data['Close'], timeperiod=20)

# Prepare the dataset
data.dropna(inplace=True)
data['Target'] = np.where(data['Close'].shift(-1) > data['Close'], 1, 0)
features = data[['SMA50', 'SMA200', 'RSI', 'UpperBB', 'LowerBB']]
labels = data['Target']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Train the model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict and evaluate
predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

# Generate buy/sell signals
data['Prediction'] = model.predict(features)
data['Buy'] = np.where(data['Prediction'] == 1, 1, 0)
data['Sell'] = np.where(data['Prediction'] == 0, -1, 0)

# Export signals to a CSV file
data[['Buy', 'Sell']].to_csv('signals.csv')

[*********************100%***********************]  1 of 1 completed


TypeError: Argument 'real' has incorrect type (expected numpy.ndarray, got DataFrame)